In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

Read data from csv file 

In [5]:
stop_file = "static/data/Police_Stop_data.csv"
stop_df=pd.read_csv(stop_file)
stop_df.head()

,OBJECTID,masterIncidentNumber,responseDate,reason,problem,callDisposition,citationIssued,personSearch,vehicleSearch,preRace,race,gender,lat,long,x,y,policePrecinct,neighborhood,lastUpdateDate
0,1001,16-406479,2016-11-09T17:43:21.000Z,NaN,Attempt Pick-Up (P),BKG-Booking,NaN,YES,NO,Native American,Native American,Male,44.949552,-93.281346,-1.038403e+07,5.613583e+06,5.0,Whittier,2017-08-08T10:24:35.000Z
1,1002,16-406495,2016-11-09T17:58:28.000Z,NaN,Suspicious Vehicle (P),ADV-Advised,NaN,NO,NO,Unknown,Black,Male,45.014157,-93.308244,-1.038703e+07,5.623750e+06,4.0,Cleveland,2017-08-08T10:24:35.000Z
2,1003,16-406507,2016-11-09T18:04:48.000Z,NaN,Suspicious Vehicle (P),GOA-Gone on Arrival,NaN,NaN,NaN,NaN,NaN,NaN,44.995210,-93.299420,-1.038604e+07,5.620767e+06,4.0,Near - North,2017-08-08T10:25:31.000Z
3,1004,16-406526,2016-11-09T18:20:12.000Z,NaN,Traffic Law Enforcement (P),NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.982470,-93.244000,-1.037987e+07,5.618762e+06,2.0,Marcy Holmes,2017-08-08T10:25:31.000Z
4,1005,16-406555,2016-11-09T18:40:01.000Z,NaN,Suspicious Vehicle (P),AOK- All OK,NaN,NO,NO,Unknown,Unknown,Unknown,44.911719,-93.216454,-1.037681e+07,5.607634e+06,3.0,Minnehaha,2017-08-08T10:25:03.000Z


# Reduce Columns
Reduce the columns to those needed and eliminate rows with null gender  
check count to confirm later operations  

In [6]:
      #select needed columns
select_df=stop_df.loc[:,["OBJECTID", "responseDate","citationIssued","lat","long","gender","neighborhood",]]

# remove null gender values
reduced_df=select_df.dropna(axis=0,subset=['gender'])
reduced_df.head()

,OBJECTID,responseDate,citationIssued,lat,long,gender,neighborhood
0,1001,2016-11-09T17:43:21.000Z,NaN,44.949552,-93.281346,Male,Whittier
1,1002,2016-11-09T17:58:28.000Z,NaN,45.014157,-93.308244,Male,Cleveland
4,1005,2016-11-09T18:40:01.000Z,NaN,44.911719,-93.216454,Unknown,Minnehaha
5,1006,2016-11-09T18:40:44.000Z,NaN,44.976087,-93.277436,Male,Downtown West
6,1007,2016-11-09T18:51:47.000Z,NaN,44.959130,-93.288030,Male,Lowry Hill East


In [7]:
#Get a listing and count of neighborhoods
neighbor=reduced_df.groupby(["neighborhood"]).count()
neighbor

,OBJECTID,responseDate,citationIssued,lat,long,gender
neighborhood,,,,,,
Armatage,154,154,118,154,154,154
Audubon Park,1217,1217,824,1217,1217,1217
Bancroft,219,219,144,219,219,219
Beltrami,441,441,294,441,441,441
Bottineau,641,641,364,641,641,641
Bryant,161,161,115,161,161,161
Bryn - Mawr,235,235,168,235,235,235
CARAG,948,948,600,948,948,948
Camden Industrial,69,69,50,69,69,69


# modify Gender 
combine gender Unknown and Gender Non-Conforming into gender Other

In [8]:
pd.options.mode.chained_assignment = None  # default='warn'
#fill in Unknown vaue with Other
reduced_df.loc[reduced_df.gender=="Unknown","gender"] = "Other"
      # fill in Gender Non-Conforming values with other
reduced_df.loc[reduced_df.gender =="Gender Non-Conforming","gender"] = "Other"
reduced_df.head()

,OBJECTID,responseDate,citationIssued,lat,long,gender,neighborhood
0,1001,2016-11-09T17:43:21.000Z,NaN,44.949552,-93.281346,Male,Whittier
1,1002,2016-11-09T17:58:28.000Z,NaN,45.014157,-93.308244,Male,Cleveland
4,1005,2016-11-09T18:40:01.000Z,NaN,44.911719,-93.216454,Other,Minnehaha
5,1006,2016-11-09T18:40:44.000Z,NaN,44.976087,-93.277436,Male,Downtown West
6,1007,2016-11-09T18:51:47.000Z,NaN,44.959130,-93.288030,Male,Lowry Hill East


In [9]:
#what gender values are present ?
#verify count
gender=reduced_df.groupby(["gender"]).count()
gender

,OBJECTID,responseDate,citationIssued,lat,long,neighborhood
gender,,,,,,
Female,19913,19913,12614,19913,19913,19717
Male,55814,55814,36553,55814,55814,55334
Other,13521,13521,8982,13521,13521,13398


# Modify citationIssued 
change NaN to NO

In [10]:
# change null citationIssued to NO
reduced_df[["citationIssued"]]=reduced_df[["citationIssued"]].fillna(value='NO')
reduced_df.head()

,OBJECTID,responseDate,citationIssued,lat,long,gender,neighborhood
0,1001,2016-11-09T17:43:21.000Z,NO,44.949552,-93.281346,Male,Whittier
1,1002,2016-11-09T17:58:28.000Z,NO,45.014157,-93.308244,Male,Cleveland
4,1005,2016-11-09T18:40:01.000Z,NO,44.911719,-93.216454,Other,Minnehaha
5,1006,2016-11-09T18:40:44.000Z,NO,44.976087,-93.277436,Male,Downtown West
6,1007,2016-11-09T18:51:47.000Z,NO,44.959130,-93.288030,Male,Lowry Hill East


In [11]:
#what gender values are present ?
# verify count
citation=reduced_df.groupby(["citationIssued"]).count()
citation

,OBJECTID,responseDate,lat,long,gender,neighborhood
citationIssued,,,,,,
NO,79906,79906,79906,79906,79906,79183
YES,9342,9342,9342,9342,9342,9266


# Parse date
responseDow = day of the week  
responseDay = day of the month as zero padded decimal  
responseDonth = month as a zero padded decimal  
responseMonthName= month as abbreviated name  
responseYear  

In [14]:
#parse date
date_list= [datetime.strptime( entry,"%Y-%m-%dT%H:%M:%S.000Z") for entry in reduced_df["responseDate"]]
# determine response day of week
dow_list= [datetime.strftime(entry,"%w") for entry in date_list]

# split off response day, month and year
day_list = [datetime.strftime(entry,"%d") for entry in date_list]
month_list=[datetime.strftime(entry,"%m") for entry in date_list]
month_name_list=[datetime.strftime(entry,"%b") for entry in date_list]
year_list= [datetime.strftime(entry,"%Y") for entry in date_list]

#add column to reduced_df
reduced_df.loc[:,"responseDow"]=dow_list
reduced_df.loc[:,"responseDay"]=day_list
reduced_df.loc[:,"responseMonth"]=month_list
reduced_df.loc[:,"responseMonthName"]=month_name_list
reduced_df.loc[:,"responseYear"]=year_list
reduced_df.head()

,OBJECTID,responseDate,citationIssued,lat,long,gender,neighborhood,responseDow,responseDay,responseMonth,responseMonthName,responseYear
0,1001,2016-11-09T17:43:21.000Z,NO,44.949552,-93.281346,Male,Whittier,3,09,11,Nov,2016
1,1002,2016-11-09T17:58:28.000Z,NO,45.014157,-93.308244,Male,Cleveland,3,09,11,Nov,2016
4,1005,2016-11-09T18:40:01.000Z,NO,44.911719,-93.216454,Other,Minnehaha,3,09,11,Nov,2016
5,1006,2016-11-09T18:40:44.000Z,NO,44.976087,-93.277436,Male,Downtown West,3,09,11,Nov,2016
6,1007,2016-11-09T18:51:47.000Z,NO,44.959130,-93.288030,Male,Lowry Hill East,3,09,11,Nov,2016


# Deliver clean stop data
to file "static/data/clean_data.csv"  
or to database

In [15]:
reduced_df.to_csv("static/data/clean_stop_data.csv", index=False,header=True)

# Prep responseDow data
get count by neighborhood, responseDow and gender

In [16]:
# get count of genders by neighborhood and responseDow
group_df=reduced_df.groupby(["neighborhood","responseDow","gender"]).count()
group_df.head()

OBJECTID  responseDate  citationIssued  lat  \
neighborhood responseDow gender                                                
Armatage     0           Female         1             1               1    1   
                         Male           9             9               9    9   
                         Other          9             9               9    9   
             1           Female         2             2               2    2   
                         Male           6             6               6    6   

                                 long  responseDay  responseMonth  \
neighborhood responseDow gender                                     
Armatage     0           Female     1            1              1   
                         Male       9            9              9   
                         Other      9            9              9   
             1           Female     2            2              2   
                         Male       6            6              6   

                                 responseMonthName  responseYear  
neighborhood responseDow gender                                   
Armatage     0           Female                  1             1  
                         Male                    9             9  
                         Other                   9             9  
             1           Female                  2             2  
                         Male                    6             6

In [17]:
# remove indexing rename columns and reduce columns
new_group=group_df.reset_index()
next_group=new_group.rename(columns={"OBJECTID":"genderCount"})
dow_group=next_group[["neighborhood","responseDow","gender","genderCount"]]
dow_group.head()

,neighborhood,responseDow,gender,genderCount
0,Armatage,0,Female,1
1,Armatage,0,Male,9
2,Armatage,0,Other,9
3,Armatage,1,Female,2
4,Armatage,1,Male,6


# Deliver clean day of week data  
to file "static/data/dow_clean_data.csv" 
or database

In [18]:
# write ad csv file -- json file doesn't come out right
dow_group.to_csv("static/data/clean_dow_data.csv" ,index=False, header=True)